In [1]:
%load_ext autoreload
%autoreload 2

In [23]:
import numpy
from collections import defaultdict
from tqdm.notebook import tqdm, trange
import pickle
import h5py

board_evals = defaultdict(lambda: [])
board_inds = defaultdict(lambda: [])
tuple_index = 0
for i in trange(16):
    with h5py.File(f"/data/chess{i}.h5", "r") as f:
        boards, evals = f["boards"], f["evaluations"]
        for board, eval in tqdm(zip(boards, evals), total=len(boards)):
            board_str = " ".join([str(x) for x in board.flatten().tolist()])
            board_evals[board_str].append(eval.item())
            board_inds[board_str].append(tuple_index)
            tuple_index += 1

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 0/1000000 [00:00<?, ?it/s]

...

  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 0/550875 [00:00<?, ?it/s]

In [24]:
valid_indices = [inds[0] for inds in board_inds.values()]
with open("valid_indices.pkl", "wb") as fp:
    pickle.dump(valid_indices, fp)

In [26]:
valid_index_evals = [evals[0] for evals in board_evals.values()]
with open("valid_index_evals.pkl", "wb") as fp:
    pickle.dump(valid_index_evals, fp)

In [25]:
len(board_inds)

13238717

In [45]:
import pickle
from tqdm.notebook import tqdm, trange
import torch
from utils.chess_primitives import *

In [43]:
def convert_labeled_game(input_game):

    # Output of function
    states = []
    evals = []

    board = init_board()
    turn = "white"

    for move, stockfish_eval in zip(input_game["moves_pgn"], input_game["eval"]):
        # Retrieve the move
        move = move.replace('x','').replace('+','').replace('#','') # Drop capture/check/checkmate notation - unnecessary
        # Revert any passant pawns to regular pawns if they survived the enemy's turn.
        board[board==2] = 1
        # Calculate legal moves from this position
        candidates = candidate_moves(board)
        # Generate candidates and identify the board of the move that was made
        played_board = get_played(board, move, turn, candidates, False)
        # Maybe terminate
        if played_board is None:
            return states, evals

        # Add updated board to list - if not checkmate
        if stockfish_eval["type"] == "cp":
            states.append(played_board.copy())
            evals.append(stockfish_eval["value"] * (1 if turn == "white" else -1))

        # Record the new board state
        board = played_board
        # Conjugate board to other side's view
        board = conjugate_board(board)
        # Turn play over to the other side.
        if turn == 'white':
            turn = 'black'
        else:
            turn = 'white'

    return states, evals

In [46]:
new_dataset = []

for i in trange(15):
    with open(f"labeled_games/{i:08}.pkl", "rb") as fp:
        data = pickle.load(fp)

    for game in tqdm(data):
        game_states, game_vals = convert_labeled_game(game)
        new_dataset += list(zip(game_states, game_vals))

    torch.save(new_dataset, "general_dataset.pt")

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/256 [00:00<?, ?it/s]

KeyboardInterrupt: 